# Scrape Texas hearing decisions



## Imports and functions

In [1]:

import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## link processing
import urllib
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests 
import itertools
import re
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [3]:
print("Loaded imports successfully")

path_write = "/Users/raj2/Dropbox/dph_hearing_decisions/data/texas/hearings/"

Loaded imports successfully


## Define scraping functions

In [4]:
def parse_https_page(link):
    
    parsed_page = BeautifulSoup(urlopen(Request(link, headers={'User-Agent': 'Mozilla/5.0'})).read(),
                    "html.parser")
    return(parsed_page)
    
def extract_links_frompage(parsed_page, pattern_tosearch):
    
    empty_list = []
    for link in parsed_page.findAll('a', attrs={'href': re.compile(pattern_tosearch)}):
        empty_list.append(link.get('href'))
    
    return(empty_list)

# Navigate pages to download hearing pdfs

Weird aspx tables so just copying site content as string

In [66]:
table_2018 = """
<div class="field field--name-body field--type-text-with-summary field--label-hidden field--item"><p>This page contains special education due process hearing decisions issued in 2018 in cases filed by a parent or school district.</p>  <table width="1004" height="272" cellspacing="3" cellpadding="5" border="1"><tbody><tr><td><p align="center"><b>Docket #</b></p>  </td>  <td><p align="center"><b>Petitioner</b></p>  </td>  <td><p align="center"><b>Respondent</b></p>  </td>  <td><p align="center"><b>Decision Date</b></p>  </td>  <td><p align="center"><b>Hearing Officer</b></p>  </td>  <td><p align="center"><b>Keywords</b></p>  </td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/021-SE-1016_Kirbyville%20CISD.pdf" title="021-SE-1016">021-SE-1016</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Kirbyville CISD</td>  <td colspan="1" style="text-align: center;">1/7/2018</td>  <td colspan="1" style="text-align: center;">Rudd</td>  <td colspan="1">Child Find, Evaluation/Reevaluation, FAPE, IEP, Procedural Safeguards, Reimbursement, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/280-SE-0817_Leander%20ISD.pdf" title="280-SE-0817">280-SE-0817</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Leander ISD</td>  <td colspan="1" style="text-align: center;">1/8/2018</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Behavior, FAPE, Parent Participation, Parental Placement, Prior Written Notice (PWN), Reimbursement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/263-SE-0717_Riesel%20ISD.pdf" title="263-SE-0717">263-SE-0717</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Riesel ISD</td>  <td colspan="1" style="text-align: center;">2/2/2018</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Comp Ed, FAPE, IEE, IEP, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/092-SE-1217_Lewisville%20ISD.pdf" title="092-SE-1217">092-SE-1217</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Lewisville ISD</td>  <td colspan="1" style="text-align: center;">2/8/2018</td>  <td colspan="1" style="text-align: center;">Rudd</td>  <td colspan="1">Behavior, Child Find, FAPE, Parent Participation, Procedural Safeguards</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/276-SE-0817_Cleveland%20ISD.pdf" title="276-SE-0817">276-SE-0817</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Cleveland ISD</td>  <td colspan="1" style="text-align: center;">3/12/2018</td>  <td colspan="1" style="text-align: center;">Spechler</td>  <td colspan="1">Comp Ed, FAPE, IEE, IEP, Parent Participation, Prior Written Notice (PWN), Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/040-SE-1017_Huntsville%20ISD.pdf" title="040-SE-1017">040-SE-1017</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Huntsville ISD</td>  <td colspan="1" style="text-align: center;">3/19/2018</td>  <td colspan="1" style="text-align: center;">Elliot</td>  <td colspan="1">Behavior, FAPE, LRE, Parent Participation, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/284-SE-0817_Dallas%20ISD.pdf" title="284-SE-0817">284-SE-0817</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Dallas ISD</td>  <td colspan="1" style="text-align: center;">3/19/2018</td>  <td colspan="1" style="text-align: center;">Lewis</td>  <td colspan="1">Behavior, FAPE, IEP, Residential Placement, Statute of Limitations, Training</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/254-SE-0617_Northwest%20ISD.pdf" title="254-SE-0617">254-SE-0617</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Northwest ISD</td>  <td colspan="1" style="text-align: center;">4/20/2018</td>  <td colspan="1" style="text-align: center;">Elliot</td>  <td colspan="1">Child Find, FAPE, IEP, Parent Participation, Prior Written Notice (PWN), Related Services</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/172-SE-0318A_Cypress-Fairbanks%20ISD.pdf" title="172-SE-0318-A">172-SE-0318-A</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Cypress-Fairbanks ISD</td>  <td colspan="1" style="text-align: center;">5/2/2018</td>  <td colspan="1" style="text-align: center;">Lewis</td>  <td colspan="1">Behavior, Discipline, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/226-SE-0517_Leander%20ISD.pdf" title="226-SE-0517">226-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Leander ISD</td>  <td colspan="1" style="text-align: center;">5/18/2018</td>  <td colspan="1" style="text-align: center;">Elliot</td>  <td colspan="1">Assistive Tech, Comp Ed, Extended School Year (ESY), FAPE, IEP, Parent Participation, Prior Written Notice (PWN), Reimbursement, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/002-SE-0917_El%20Paso%20ISD.pdf" title="002-SE-0917">002-SE-0917</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">El Paso ISD</td>  <td colspan="1" style="text-align: center;">5/22/2018</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Behavior, Child Find, FAPE, IEP, Residential Placement, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/255-SE-0617_Judson%20ISD.pdf" title="255-SE-0617">255-SE-0617</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Judson ISD</td>  <td colspan="1" style="text-align: center;">6/4/2018</td>  <td colspan="1" style="text-align: center;">Spechler</td>  <td colspan="1">Assistive Tech, Comp Ed, Extended School Year (ESY), FAPE, IEP, LRE, Parent Participation, Procedural Safeguards, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/146-SE-0218_Klein%20ISD.pdf" title="146-SE-0218">146-SE-0218</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Klein ISD</td>  <td colspan="1" style="text-align: center;">6/25/2018</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">IEE, IEP, LRE</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/148-SE-0317_Waxahachie%20ISD.pdf" title="148-SE-0317">148-SE-0317</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Waxahachie ISD</td>  <td colspan="1" style="text-align: center;">7/17/2018</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Behavior, Comp Ed, FAPE, IEE, IEP, Parent Participation, Reimbursement, Training</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/121-SE-0118_Austin%20ISD.pdf" title="121-SE-0118">121-SE-0118</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Austin ISD</td>  <td colspan="1" style="text-align: center;">8/10/2018</td>  <td colspan="1" style="text-align: center;">Green</td>  <td colspan="1">FAPE, IEE, IEP, LRE, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/035-SE-1017_Leander%20ISD.pdf" title="035-SE-1017">035-SE-1017</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Leander ISD</td>  <td colspan="1" style="text-align: center;">9/12/2018</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Behavior, FAPE, IEP, Jurisdiction, LRE, Parent Participation, Placement, Reimbursement, Residential Placement, Transfer Students</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/186-SE-0417_Austin%20ISD.pdf" title="186-SE-0417">186-SE-0417</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Austin ISD</td>  <td colspan="1" style="text-align: center;">9/19/2018</td>  <td colspan="1" style="text-align: center;">Rudd</td>  <td colspan="1">Assistive Tech, Behavior, Comp Ed, Extended School Year (ESY), FAPE, LRE, Reimbursement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/241-SE-0518_Florence%20ISD.pdf" title="241-SE-0518">241-SE-0518</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Florence ISD</td>  <td colspan="1" style="text-align: center;">10/29/2018</td>  <td colspan="1" style="text-align: center;">Lewis</td>  <td colspan="1">Behavior, Comp Ed, Consent, FAPE, LRE, Parent Participation, Procedural Safeguards, Prior Written Notice (PWN), Residential Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/224-SE-0418_Klein%20ISD.pdf" title="224-SE-0418">224-SE-0418</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Klein ISD</td>  <td colspan="1" style="text-align: center;">11/16/2018</td>  <td colspan="1" style="text-align: center;">Spechler</td>  <td colspan="1">Child Find, Dyslexia, Eligibility, FAPE, Procedural Safeguards, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/185-SE-0418_Conroe%20ISD.pdf" title="185-SE-0418">185-SE-0418</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Conroe ISD</td>  <td colspan="1" style="text-align: center;">11/19/2018</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">FAPE, IEP, Reimbursement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/062-SE-1118_Kirbyville%20CISD.pdf" title="062-SE-1118">062-SE-1118</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Kirbyville CISD</td>  <td colspan="1" style="text-align: center;">12/14/2018</td>  <td colspan="1" style="text-align: center;">Green</td>  <td colspan="1">Accomodations, Behavior, Child Find, Discipline, Eligibility, Evaluation/Reevaluation, FAPE</td>  </tr></tbody></table><h5>Due Process Program Questions: <a href="mailto:Brittney.Salaiz@tea.texas.gov?subject=2017 DPH Question" title="2017 DPH Question">Brittney Salaiz</a></h5>  </div>
      

"""
split_table = table_2018.split(" ")
links = [re.sub('href=|"', "", str(link)) for link in split_table if "href" in link and 
        "sites" in link]

links_topull = ["https://tea.texas.gov" + str(link) for link in links]
links_topull

## iterate through links and download
for hod in links_topull:
    resp = requests.get(hod, verify=False,stream=True)
    filename = path_write + re.sub('https.*/sites/default/files/', '', hod) 
    fd = open(filename, 'wb')
    fd.write(resp.content)
    fd.close() 

['https://tea.texas.gov/sites/default/files/021-SE-1016_Kirbyville%20CISD.pdf',
 'https://tea.texas.gov/sites/default/files/280-SE-0817_Leander%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/263-SE-0717_Riesel%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/092-SE-1217_Lewisville%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/276-SE-0817_Cleveland%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/040-SE-1017_Huntsville%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/284-SE-0817_Dallas%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/254-SE-0617_Northwest%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/172-SE-0318A_Cypress-Fairbanks%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/226-SE-0517_Leander%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/002-SE-0917_El%20Paso%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/255-SE-0617_Judson%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/146-SE-0218_Klein%20

183606

189763

261366

113224

...

151591

255147

215939

40892

287154

In [68]:
table_2017 = """<div class="field field--name-body field--type-text-with-summary field--label-hidden field--item"><p>This page contains special education due process hearing decisions issued in 2017 in cases filed by a parent or school district.</p>  <table width="1004" cellspacing="3" cellpadding="5" border="1" height="272"><tbody><tr><td><p align="center"><b>Docket #</b>           </p>  </td>  <td><p align="center"><b>Petitioner</b>           </p>  </td>  <td><p align="center"><b>Respondent</b>           </p>  </td>  <td><p align="center"><b>Decision Date</b>           </p>  </td>  <td><p align="center"><b>Hearing Officer</b>           </p>  </td>  <td><p align="center"><b>Keywords</b>           </p>  </td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/009-SE-0916_Huntsville%20ISD.pdf" title="009-SE-0916">009-SE-0916</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Huntsville ISD</td>  <td colspan="1" style="text-align: center;">1/13/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">FAPE, IEP, LRE</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/056-SE-1116_Round%20Rock%20ISD.pdf" title="056-SE-1116">056-SE-1116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Round Rock ISD</td>  <td colspan="1" style="text-align: center;">1/31/2017</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">Procedural Safeguards</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/316-SE-0716_Quinlan%20ISD.pdf" title="316-SE-0716">316-SE-0716</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Quinlan ISD</td>  <td colspan="1" style="text-align: center;">3/7/2017</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">Behavior, Evaluation/Reevaluation, FAPE, IEP, LRE, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/016-SE-0916_Abilene%20ISD.pdf" title="016-SE-0916">016-SE-0916</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Abilene ISD</td>  <td colspan="1" style="text-align: center;">3/14/2017</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Behavior, Evaluation/Reevaluation, FAPE, IEP, Residential Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/223-SE-0416_Leander%20ISD.pdf" title="223-SE-0416">223-SE-0416</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Leander ISD</td>  <td colspan="1" style="text-align: center;">3/27/2017</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Child Find</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/072-SE-1116_Clear%20Creek%20ISD.pdf" title="072-SE-1116">072-SE-1116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Clear Creek ISD</td>  <td colspan="1" style="text-align: center;">4/7/2017</td>  <td colspan="1" style="text-align: center;">Egan</td>  <td colspan="1">Evaluation/Reevaluation, FAPE, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/062-SE-1116_Georgetown%20ISD.pdf" title="062-SE-1116">062-SE-1116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Georgetown ISD</td>  <td colspan="1" style="text-align: center;">4/10/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Evaluation/Reevaluation, FAPE, IEP, Parental Placement, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/069-SE-1116_Uplift%20Education.pdf" title="069-SE-1116">069-SE-1116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Uplift Education</td>  <td colspan="1" style="text-align: center;">4/21/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Assistive Tech, Behavior, FAPE, IEP, Methodology, Related Services</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/030-SE-1016_Cedar%20Hill%20ISD.pdf" title="030-SE-1016">030-SE-1016</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Cedar Hill ISD</td>  <td colspan="1" style="text-align: center;">4/24/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">FAPE, IEE, IEP, Related Services</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/183-SE-0417_Klein%20ISD.pdf" title="183-SE-0417">183-SE-0417</a></td>  <td colspan="1" style="text-align: center;">Klein ISD</td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">5/10/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Discipline, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/286-SE-0616_Killeen%20ISD.pdf" title="286-SE-0616">286-SE-0616</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Killeen ISD</td>  <td colspan="1" style="text-align: center;">5/23/2017</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">Behavior, Child Find, Eligibility, FAPE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/098-SE-0117_North%20East%20ISD.pdf" title="098-SE-0117">098-SE-0117</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">North East ISD</td>  <td colspan="1" style="text-align: center;">6/30/2017</td>  <td colspan="1" style="text-align: center;">Egan</td>  <td colspan="1">Child Find</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/101-SE-0117_Copperas%20Cove%20ISD.pdf" title="101-SE-0117">101-SE-0117</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Copperas Cove ISD</td>  <td colspan="1" style="text-align: center;">6/30/2017</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">Child Find, Evaluation/Reevaluation, FAPE, IEE, IEP, Procedural Safeguards</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/092-SE-1216_Riesel%20ISD.pdf" title="092-SE-1216">092-SE-1216</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Riesel ISD</td>  <td colspan="1" style="text-align: center;">7/6/2017</td>  <td colspan="1" style="text-align: center;">Wetsch</td>  <td colspan="1">Assistive Tech, Behavior, Eligibility, FAPE, IEP, LRE, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/179-SE-0317_Clear%20Creek%20ISD.pdf" title="179-SE-0317">179-SE-0317</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Clear Creek ISD</td>  <td colspan="1" style="text-align: center;">7/7/2017</td>  <td colspan="1" style="text-align: center;">Egan</td>  <td colspan="1">Assistive Tech, FAPE, IEP, LRE</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/156-SE-0317_Lancaster%20ISD.pdf" title="156-SE-0317">156-SE-0317</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Lancaster ISD</td>  <td colspan="1" style="text-align: center;">7/7/2017</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Behavior, Eligibility, FAPE, IEE, IEP, LRE, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/001-SE-0916_North%20East%20ISD.pdf" title="001-SE-0916">001-SE-0916</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">North East ISD</td>  <td colspan="1" style="text-align: center;">7/13/2017</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Behavior, FAPE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/014-SE-0916_Killeen%20ISD.pdf" title="014-SE-0916">014-SE-0916</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Killeen ISD</td>  <td colspan="1" style="text-align: center;">7/15/2017</td>  <td colspan="1" style="text-align: center;">Wetsch</td>  <td colspan="1">Assistive Tech, Behavior, FAPE, IEE, IEP, LRE, Parental Placement, Procedural Safeguards, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/157-SE-0317_Lubbock%20Cooper%20ISD.pdf" title="157-SE-0317">157-SE-0317</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Lubbock-Cooper ISD</td>  <td colspan="1" style="text-align: center;">7/31/2017</td>  <td colspan="1" style="text-align: center;">Wetsch</td>  <td colspan="1">Eligibility, FAPE, IEP, LRE, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/117-SE-0217_Pearland%20ISD.pdf" title="117-SE-0217">117-SE-0217</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Pearland ISD</td>  <td colspan="1" style="text-align: center;">8/1/2017</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Assistive Tech, Behavior, Child Find, Evaluation/Reevaluation, FAPE, IEP, LRE, Parental Placement, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/195-SE-0417_Port%20Arthur%20ISD.pdf" title="195-SE-0417">195-SE-0417</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Port Arthur ISD</td>  <td colspan="1" style="text-align: center;">8/15/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">FAPE, IEP, Placement, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/224-SE-0517_Smithville%20ISD.pdf" title="224-SE-0517">224-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Smithville ISD</td>  <td colspan="1" style="text-align: center;">8/18/2017</td>  <td colspan="1" style="text-align: center;">Elliot</td>  <td colspan="1">Behavior, Discipline</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/158-SE-0317_Lubbock-Cooper%20ISD.pdf" title="158-SE-0317">158-SE-0317</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Lubbock-Cooper ISD</td>  <td colspan="1" style="text-align: center;">8/25/2017</td>  <td colspan="1" style="text-align: center;">Wetsch</td>  <td colspan="1">FAPE, IEP, LRE, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/223-SE-0517_El%20Paso%20ISD.pdf" title="223-SE-0517">223-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">El Paso ISD</td>  <td colspan="1" style="text-align: center;">9/25/2017</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Behavior, FAPE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/099-SE-0117_Northside%20ISD.pdf" title="099-SE-0117">099-SE-0117</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Northside ISD</td>  <td colspan="1" style="text-align: center;">10/4/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">ARD, Assistive Tech, Extended School Year (ESY), FAPE, IEP, Methodology, Transfer Students</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/250-SE-0617_Argyle%20ISD.pdf" title="250-SE-0617">250-SE-0617</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Argyle ISD</td>  <td colspan="1" style="text-align: center;">10/12/2017</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">FAPE, IEP, LRE</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/248-SE-0617_Houston%20ISD.pdf" title="248-SE-0617">248-SE-0617</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Houston ISD</td>  <td colspan="1" style="text-align: center;">10/24/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Child Find, Parental Placement, Statute of Limitations</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/218-SE-0517_Northside%20ISD.pdf" title="218-SE-0517">218-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Northside ISD</td>  <td colspan="1" style="text-align: center;">10/27/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Behavior, FAPE, IEP, Parent Participation, Residential Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/009-SE-0917_Riesel%20ISD.pdf" title="009-SE-0917">009-SE-0917</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Riesel ISD</td>  <td colspan="1" style="text-align: center;">10/30/2017</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Discipline</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/027-SE-1017_Conroe%20ISD.pdf" title="027-SE-1017">027-SE-1017</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Conroe ISD</td>  <td colspan="1" style="text-align: center;">11/15/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Discipline</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/214-SE-0517_Vernon%20ISD.pdf" title="214-SE-0517">214-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Vernon ISD</td>  <td colspan="1" style="text-align: center;">11/17/2017</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Discipline, Eligibility, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/208-SE-0517_Spring%20Branch%20ISD.pdf" title="208-SE-0517">208-SE-0517</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Spring Branch ISD</td>  <td colspan="1" style="text-align: center;">12/6/2017</td>  <td colspan="1" style="text-align: center;">Elliot</td>  <td colspan="1">Extended School Year (ESY), FAPE, Parental Placement, Reimbursement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/249-SE-0617_Pearland%20ISD.pdf" title="249-SE-0617">249-SE-0617</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Pearland ISD</td>  <td colspan="1" style="text-align: center;">12/8/2017</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Child Find, IEP, Parental Placement, Placement, Procedural Safeguards, Reimbursement, Statute of Limitation</td>  </tr></tbody></table><h5>Due Process Program Questions: <a href="mailto:Brittney.Salaiz@tea.texas.gov?subject=2017 DPH Question" title="2017 DPH Question">Brittney Salaiz</a></h5>  </div>"""

split_table = table_2017.split(" ")
links = [re.sub('href=|"', "", str(link)) for link in split_table if "href" in link and 
        "sites" in link]

links_topull = ["https://tea.texas.gov" + str(link) for link in links]
links_topull

## iterate through links and download
for hod in links_topull:
    resp = requests.get(hod, verify=False,stream=True)
    filename = path_write + re.sub('https.*/sites/default/files/', '', hod) 
    fd = open(filename, 'wb')
    fd.write(resp.content)
    fd.close() 

['https://tea.texas.gov/sites/default/files/009-SE-0916_Huntsville%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/056-SE-1116_Round%20Rock%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/316-SE-0716_Quinlan%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/016-SE-0916_Abilene%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/223-SE-0416_Leander%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/072-SE-1116_Clear%20Creek%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/062-SE-1116_Georgetown%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/069-SE-1116_Uplift%20Education.pdf',
 'https://tea.texas.gov/sites/default/files/030-SE-1016_Cedar%20Hill%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/183-SE-0417_Klein%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/286-SE-0616_Killeen%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/098-SE-0117_North%20East%20ISD.pdf',
 'https://tea.texas.gov/sites/default/files/101-SE-0117_Co

38049

74136

317863

240751

...

111108

32242

175762

134672

58123

In [69]:
table_2016 = """<div class="field field--name-body field--type-text-with-summary field--label-hidden field--item"><p>This page contains special education due process hearing   decisions issued in 2016 in cases filed by a parent or school district.</p>  <table width="1004" cellspacing="3" cellpadding="5" border="1" height="272"><tbody><tr><td><p align="center"><b>Docket   #</b>           </p>  </td>  <td><p align="center"><b>Petitioner</b>           </p>  </td>  <td><p align="center"><b>Respondent</b>           </p>  </td>  <td><p align="center"><b>Decision Date</b>           </p>  </td>  <td><p align="center"><b>Hearing Officer</b>           </p>  </td>  <td><p align="center"><b>Keywords</b>           </p>  </td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/369-SE-0815_Bastrop.pdf" title="369-SE-0815">369-SE-0815</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Bastrop ISD</td>  <td colspan="1" style="text-align: center;">1/15/2016</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">ARD, Evaluation/Reevaluation, FAPE, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/WorkArea/linkit.aspx?LinkIdentifier=id&amp;ItemID=25769825537" title="201-SE-0315">201-SE-0315</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Houston ISD</td>  <td colspan="1" style="text-align: center;">2/3/2016</td>  <td colspan="1" style="text-align: center;">Rubinett</td>  <td colspan="1">FAPE, IEP, LRE, Parent Participation, Parental Placement, Reimbursement, Statute of Limitation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/355-SE-0815_Houston.pdf" title="355-SE-0815">355-SE-0815</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Houston ISD</td>  <td colspan="1" style="text-align: center;">2/12/2016</td>  <td colspan="1" style="text-align: center;">Rudd</td>  <td colspan="1">FAPE, IEP, Parental Placement, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/017-SE-0915_Pflugerville.pdf" title="017-SE-0915">017-SE-0915</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Pflugerville ISD</td>  <td colspan="1" style="text-align: center;">2/19/2016</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Parental Placement, Residential Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/024-SE-0915_Gregory-Portland.pdf" title="024-SE-0915">024-SE-0915</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Gregory Portland ISD</td>  <td colspan="1" style="text-align: center;">3/25/2016</td>  <td colspan="1" style="text-align: center;">Rubinett</td>  <td colspan="1">FAPE, Related Services</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/166-SE-0216_Allen.pdf" title="166-SE-0216">166-SE-0216</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Allen ISD</td>  <td colspan="1" style="text-align: center;">4/13/2016</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Discipline, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/136-SE-0116_Killeen1.pdf" title="136-SE-0116">136-SE-0116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Killeen ISD</td>  <td colspan="1" style="text-align: center;">4/25/2016</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Eligibility, FAPE, IEE, IEP, LRE, Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/102-SE-1115_Riesel.pdf" title="102-SE-1115">102-SE-1115</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Riesel ISD</td>  <td colspan="1" style="text-align: center;">5/23/2016</td>  <td colspan="1" style="text-align: center;">Carmichael</td>  <td colspan="1">Child Find, Evaluation/Reevaluation, FAPE, IEP, Procedural Safeguards, Prior Written Notice</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/129-SE-0116_Bullard.pdf" title="129-SE-0116">129-SE-0116</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Bullard ISD</td>  <td colspan="1" style="text-align: center;">6/24/2016</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Evaluation/Reevaluation, FAPE, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/175-SE-0316_Warren.pdf" title="175-SE-0316">175-SE-0316</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Warren ISD</td>  <td colspan="1" style="text-align: center;">7/5/2016</td>  <td colspan="1" style="text-align: center;">Carmichael</td>  <td colspan="1">Behavior, Evaluation/Reevaluation, FAPE, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/219-SE-0415_HighlandPark.pdf" title="219-SE-0415">219-SE-0415</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Highland Park ISD</td>  <td colspan="1" style="text-align: center;">7/19/2016</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Assistive Tech, FAPE, IEE, IEP, LRE, Parent Participation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/144-SE-0216_Galveston.pdf" title="144-SE-0216">144-SE-0216</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Galveston ISD</td>  <td colspan="1" style="text-align: center;">7/29/2016</td>  <td colspan="1" style="text-align: center;">Lockwood</td>  <td colspan="1">Behavior, Evaluation/Reevaluation, FAPE, IEP, Residential Placement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/068-SE-1015_SpringBranch.pdf" title="068-SE-1015">068-SE-1015</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Spring Branch ISD</td>  <td colspan="1" style="text-align: center;">8/3/2016</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">FAPE, IEP, LRE, Placement, Prior Written Notice (PWN), Reimbursement</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/111-SE-1215_Houston.pdf" title="111-SE-1215">111-SE-1215</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Houston ISD</td>  <td colspan="1" style="text-align: center;">8/15/2016</td>  <td colspan="1" style="text-align: center;">Bunton</td>  <td colspan="1">Evaluation/Reevaluation, FAPE, IEE, IEP</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/267-SE-0516_LeanderISD.pdf" title="267-SE-0516">267-SE-0516</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Leander ISD</td>  <td colspan="1" style="text-align: center;">8/22/2016</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Eligibility, Evaluation/Reevaluation, IEE, IEP, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/312-SE-0716_NorthsideISD.pdf" title="312-SE-0716">312-SE-0716</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Northside ISD</td>  <td colspan="1" style="text-align: center;">12/5/2016</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">Evaluation/Reevaluation, FAPE, IEE, IEP, Prior Written Notice (PWN)</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/332-SE-0816_Tomball.pdf" title="332-SE-0816">332-SE-0816</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Tomball ISD</td>  <td colspan="1" style="text-align: center;">12/6/2016</td>  <td colspan="1" style="text-align: center;">Berger</td>  <td colspan="1">FAPE, IEP, LRE, Parent Participation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/305-SE-0616_Conroe.pdf" title="305-SE-0616">305-SE-0616</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Conroe ISD</td>  <td colspan="1" style="text-align: center;">12/16/2016</td>  <td colspan="1" style="text-align: center;">Broyles</td>  <td colspan="1">Eligibility, IEE, IEP, LRE, Parent Participation</td>  </tr><tr><td colspan="1" style="text-align: center;"><a href="/sites/default/files/024-SE-1016_Northside.pdf" title="024-SE-1016">024-SE-1016</a></td>  <td colspan="1" style="text-align: center;">Student</td>  <td colspan="1" style="text-align: center;">Northside ISD</td>  <td colspan="1" style="text-align: center;">12/29/2016</td>  <td colspan="1" style="text-align: center;">Cloninger</td>  <td colspan="1">Behavior, Extended School Year (ESY), FAPE, IEP, LRE, Parent Participation</td>  </tr></tbody></table><h5>Due Process Program Questions: <a href="mailto:Brittney.Salaiz@tea.texas.gov?subject=2016 DPH Question" title="2016 DPH Question">Brittney Salaiz</a></h5>  <p> </p>  </div>"""

In [70]:
split_table = table_2016.split(" ")
links = [re.sub('href=|"', "", str(link)) for link in split_table if "href" in link and 
        "sites" in link]

links_topull = ["https://tea.texas.gov" + str(link) for link in links]
links_topull

## iterate through links and download
for hod in links_topull:
    resp = requests.get(hod, verify=False,stream=True)
    filename = path_write + re.sub('https.*/sites/default/files/', '', hod) 
    fd = open(filename, 'wb')
    fd.write(resp.content)
    fd.close() 

['https://tea.texas.gov/sites/default/files/369-SE-0815_Bastrop.pdf',
 'https://tea.texas.gov/sites/default/files/355-SE-0815_Houston.pdf',
 'https://tea.texas.gov/sites/default/files/017-SE-0915_Pflugerville.pdf',
 'https://tea.texas.gov/sites/default/files/024-SE-0915_Gregory-Portland.pdf',
 'https://tea.texas.gov/sites/default/files/166-SE-0216_Allen.pdf',
 'https://tea.texas.gov/sites/default/files/136-SE-0116_Killeen1.pdf',
 'https://tea.texas.gov/sites/default/files/102-SE-1115_Riesel.pdf',
 'https://tea.texas.gov/sites/default/files/129-SE-0116_Bullard.pdf',
 'https://tea.texas.gov/sites/default/files/175-SE-0316_Warren.pdf',
 'https://tea.texas.gov/sites/default/files/219-SE-0415_HighlandPark.pdf',
 'https://tea.texas.gov/sites/default/files/144-SE-0216_Galveston.pdf',
 'https://tea.texas.gov/sites/default/files/068-SE-1015_SpringBranch.pdf',
 'https://tea.texas.gov/sites/default/files/111-SE-1215_Houston.pdf',
 'https://tea.texas.gov/sites/default/files/267-SE-0516_LeanderISD.

676643

204278

54472

101286

...

271235

439477

266972

162509

259618

In [78]:
## 2015 and 2014 had aspx issue so hand downloaded

['CISD</td>',
 'CISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD',
 'CISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'CISD</td>',
 'CISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>',
 'ISD</td>']